<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Tipo-Test" data-toc-modified-id="Tipo-Test-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tipo Test</a></span></li></ul></div>

## Tipo Test

In [1]:
import pandas as pd
import edhec_risk_kit as erk
import plotly 
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import *

# Modo desarrollo del módulo edhec_risk_kit
%matplotlib inline
%load_ext autoreload
%autoreload 2
plotly.offline.init_notebook_mode(connected = True)

__Utilice el conjunto de datos de los Índices de Fondos de Cobertura EDHEC que utilizamos en la tarea de laboratorio, así como en las tareas de la semana anterior. Cárguelos en Python y realice el siguiente análisis basado en los datos desde el año 2000 (incluyendo todo el año 2000)__

In [2]:
fund_ind = erk.get_returns(filename="data/edhec-hedgefundindices.csv")
fund_ind = fund_ind['2000':]
fund_ind

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
date,,,,,,,,,,,,,
2000-01,0.0227,0.0128,0.0088,0.0077,0.0075,0.0088,0.0041,0.0021,0.0075,0.0143,0.0173,0.0427,0.0169
2000-02,0.0267,-0.0022,0.0421,0.0528,0.0253,0.0346,0.0097,0.0408,0.0699,0.0239,0.0185,-0.1340,0.0666
2000-03,0.0243,-0.0138,0.0103,0.0318,0.0134,0.0069,-0.0061,-0.0104,0.0006,0.0131,0.0163,-0.0230,0.0039
2000-04,0.0223,-0.0241,-0.0101,-0.0541,0.0168,-0.0059,-0.0006,-0.0304,-0.0201,0.0188,0.0092,0.1028,-0.0269
2000-05,0.0149,0.0114,-0.0132,-0.0433,0.0062,-0.0034,0.0107,-0.0070,-0.0097,0.0146,0.0080,0.0704,-0.0122
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07,0.0021,-0.0058,0.0093,0.0040,-0.0010,0.0055,0.0022,-0.0014,0.0067,-0.0021,0.0045,-0.0052,0.0018
2018-08,0.0024,0.0166,0.0002,-0.0277,0.0004,0.0011,0.0017,-0.0007,0.0035,0.0050,-0.0002,-0.0214,0.0015
2018-09,0.0034,-0.0054,0.0050,-0.0110,-0.0016,0.0032,0.0036,0.0006,-0.0023,0.0028,0.0018,0.0036,-0.0022


__Pregunta 1. ¿Cuál fue el VaR gaussiano paramétrico mensual al nivel del 1% (como número +ve) de la estrategia de valores en dificultades?__

In [3]:
df_var = erk.var_analysis(fund_ind, level=1)

Respuesta: 3.3

__Pregunta 2 Utilice el mismo conjunto de datos de la pregunta anterior. ¿Cuál fue el VaR del 1% para la misma estrategia después de aplicar el ajuste de Cornish-Fisher?__

Respuesta: 4.97

__Pregunta 3 Utilice el mismo conjunto de datos que la pregunta anterior. ¿Cuál fue el VaR histórico mensual al nivel del 1% (como número +ve) de la estrategia Distressed Securities?__

Respuesta 4.26

__A continuación, cargue los datos de las 30 rentabilidades de la industria. Para los propósitos de las preguntas restantes, utilice los datos durante el período de 5 años 2013-2017 (ambos inclusive) para estimar los rendimientos esperados, así como la matriz de covarianza. Para poder responder a las preguntas, deberá construir las carteras MSR, EW y GMV compuestas por las industrias "Books", "Steel", "Oil" y "Mines". Suponga que el tipo de interés libre de riesgo durante el periodo de 5 años es del 10%.__

In [4]:
l = ["Books", "Steel", "Oil", "Mines"]

# Lectura de datos
ind_rets = erk.get_returns(filename="data/ind30_m_vw_rets.csv")[l]
ind_rets = ind_rets['2013':'2017']

# Retorno y volatilidad anualizada
er  = erk.annualize_rets(ind_rets, 12)
cov = ind_rets.cov()

In [5]:
ind_rets.head()

,Books,Steel,Oil,Mines
2013-01,0.0513,0.0428,0.0788,0.0059
2013-02,-0.0654,-0.0268,0.0052,-0.0756
2013-03,0.0778,0.0210,0.0209,0.0091
2013-04,-0.0029,-0.0441,-0.0129,-0.1057
2013-05,0.0479,0.0384,0.0307,0.0022


In [6]:
er

Books    0.095312
Steel    0.100716
Oil      0.025275
Mines    0.005136
dtype: float64

In [7]:
cov

,Books,Steel,Oil,Mines
Books,0.002720,0.001940,0.001637,0.001539
Steel,0.001940,0.004391,0.002122,0.003391
Oil,0.001637,0.002122,0.002697,0.002179
Mines,0.001539,0.003391,0.002179,0.005232


In [8]:
# Visualización
erk.plot_ef(n_points = 20, 
            expected_returns = er, 
            covmat = cov, 
            title='Efficient Frontier Portfolio Maximum Sharpe Ratio', 
            show_cml=True, 
            risk_free_rate=0.1,
            show_ew=True,
            show_gmv=True)

__Pregunta 4 ¿Cuál es el peso del acero en la cartera EW?__

Respuesta: 25%

__Pregunta 5 ¿Cuál es el peso del mayor componente de la cartera de MSR?__

In [12]:
w_msr = erk.msr(0.1, np.array(er), cov)
ret_msr = erk.portfolio_return(w_msr, er)
vol_msr = erk.portfolio_volatility(w_msr, cov)




msr = pd.DataFrame(columns=['Assets', 'Weights'])
msr['Assets'] = er.index.to_list()
msr['Weights'] = w_msr
msr.sort_values('Weights', ascending=False, inplace=True)
msr.reset_index(drop=True, inplace=True)
msr

,Assets,Weights
0,Steel,1.000000e+00
1,Books,7.015222e-15
2,Oil,1.916869e-16
3,Mines,0.000000e+00


Respuesta: 100%


__Pregunta 6 ¿Cuál de los 4 componentes tiene el mayor peso en la cartera de MSR?__

Respuesta: Steel

__Pregunta 7 ¿Cuántos de los componentes de la cartera MSR tienen ponderaciones distintas de cero?__

Respuesta: 1


__Pregunta 8 ¿Cuál es el peso del mayor componente de la cartera de GMV?__

In [13]:
w_gmv = erk.gmv(cov)
ret_gmv = erk.portfolio_return(w_gmv, er)
vol_gmv = erk.portfolio_volatility(w_gmv, cov)


gmv = pd.DataFrame(columns=['Assets', 'Weights'])
gmv['Assets'] = er.index.to_list()
gmv['Weights'] = w_gmv
gmv.sort_values('Weights', ascending=False, inplace=True)
gmv.reset_index(drop=True, inplace=True)
gmv

,Assets,Weights
0,Books,0.477025
1,Oil,0.434112
2,Mines,0.088863
3,Steel,0.000000


Respuesta: 47.70%

__Pregunta 9 ¿Cuál de los 4 componentes tiene mayor peso en la cartera de GMV?__

Respuesta: Books

__Pregunta 10.  ¿Cuántos de los componentes de la cartera de GMV tienen ponderaciones distintas de cero?__

Respuesta: 3

__Pregunta 11. Supongamos que dos inversores diferentes invierten en las carteras GMV y MSR a principios de 2018 utilizando las ponderaciones que acabamos de calcular. Calcule la volatilidad anualizada de estas dos carteras durante los siguientes 12 meses de 2018. (Sugerencia: Utilice el código portfolio_vol que desarrollamos en el laboratorio y utilice ind["2018"][l].cov() para calcular la matriz de covarianza para 2018, suponiendo que la variable ind contiene los rendimientos de la industria y la variable l contiene la lista de carteras de la industria que usted está dispuesto a mantener. No olvides anualizar la volatilidad)
¿Cuál sería la volatilidad anualizada a lo largo de 2018 utilizando las ponderaciones de la cartera MSR?__

In [27]:
l = ["Books", "Steel", "Oil", "Mines"]

# Lectura de datos
ind_rets = erk.get_returns(filename="data/ind30_m_vw_rets.csv")[l]
ind_rets = ind_rets['2018']

# Retorno y volatilidad anualizada
er  = erk.annualize_rets(ind_rets, 12)
cov = ind_rets.cov()

In [31]:
# Visualización
erk.plot_ef(n_points = 20, 
            expected_returns = er, 
            covmat = cov, 
            title='Efficient Frontier Portfolio Maximum Sharpe Ratio', 
            show_cml=True, 
            risk_free_rate=0.1,
            show_ew=True,
            show_gmv=True)

In [28]:
erk.portfolio_volatility(w_msr, ind_rets.cov())*(12**(1/2))

0.2198398818148412

Respuesta: 21.98%

__Pregunta 12. ¿Cuál sería la volatilidad anualizada a lo largo de 2018 utilizando las ponderaciones de la cartera GMV? (Recordatorio y pista: Utiliza el código portfolio_vol que desarrollamos en el laboratorio y utiliza ind["2018"][l].cov() para calcular la matriz de covarianza para 2018, suponiendo que la variable ind contiene los rendimientos del sector y la variable l contiene la lista de carteras del sector que estás dispuesto a mantener. No olvides anualizar la volatilidad)__

In [30]:
erk.portfolio_volatility(w_gmv, ind_rets.cov())*(12**(1/2))

0.18973734530482092

Respuesta: 18.97%